# Case Study 1: Modeling (part 2)

In [ ]:
%load_ext autoreload
%autoreload 2
import sys

sys.path.append("../python")

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sys

sys.path.append("../python")

In [ ]:
from mll1 import car_pricing

In [ ]:
df_cars_raw = car_pricing.load_dataset("../../data/car_price_dataset/")
df_cars = car_pricing.prepare_dataset(df_cars_raw)

In [ ]:
quality_columns = [c for c in df_cars.columns if c.endswith("_quality")]
mask_ok_quality = (df_cars[quality_columns] == "ok").all(1)
df_cars = df_cars[mask_ok_quality].drop(quality_columns, axis=1)

In [ ]:
df_cars.info()

## Dealing with categories

Until now we have only used the numerical features of the dataset, and ignored the categorical ones.
Categorical variables are common in tabular datasets. However, most machine learning algorithms are designed to work with float values.  So in order to make use of the categorical features, we need to **convert them into numerical values** in a way that preserve their information. This conversion is usually called **categorical feature encoding**. Another option is to use algorithms specifically designed to deal with categorical data.  We will see common examples of both approaches.

### Rare categories and unknown categories

Some categories might occur rarely; this leads to two related problems:

* If the model only learns a specific behavior for a given category, it might not generalize well
* Some categories may even be completely unseen at training time, leading to a model which cannot be used at all when those values are seen at prediction time.

**Example**: Consider a prediction task depending on the geographical origin encoded as the **post code**. There are many post codes and **some post codes** associated small populations **may not occur in our training set**.

So we always need to consider those possibilities when designing our preprocessing pipeline. We will see how these problems are dealt with by the most frequent encoding strategies.

Let's prepare the dataset by:

* including categorical features;
* excluding rows with missing values.

In [ ]:
from sklearn.model_selection import train_test_split

mask_non_missing = ~df_cars["tax_mpg_missing"]

X = df_cars[mask_non_missing].drop(["price", "tax_mpg_missing"], axis=1)
y = df_cars[mask_non_missing].price.values

cat_cols = [c for c in X.columns if X.dtypes[c] == "object"]
num_cols = [c for c in X.columns if X.dtypes[c] != "object"]

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=0)

### One-hot encoding

One-hot encoding, sometimes also called dummy encoding, consists in converting a categorical variable with $C$ possible categories into $C$ binary variables.

Unseen and missing values are easily dealed with by putting all binary variables to 0.

This encoding is most often used with **linear models**.

One-hot encoding will create a feature for each category of each categorical variable. This can greatly increase the number of features. This will prevent us from using `PolynomiaFeatures` on the categorical variables.

If we still want to add non-linearities on the numerical features, will need to process categorical and numerical features separately.

Scikit-learn proposes the `sklearn.compose.ColumnTransformer` objects which allow to declare different transformers for different subsets of columns. Those transformers are declared as `(name, transformer, columns)` tuples.

```python

# Transformers/Pipelines for sub-groups of columns
preproc_1 = ...
preproc_2 = ...
preproc_3 = ...

# Use ColumnTransformer to assemble the preprocessings
transf = ColumnTransformer([
    ("preproc1", preproc_1, cols1),
    ("preproc2", preproc_2, cols2),
    ("preproc3", preproc_3, cols3)
])
```

**Exercise**: Complete the following code with a `ColumnTransformer` to assemble the preprocessing steps:

```python

from category_encoders import OneHotEncoder
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.inspection import permutation_importance


cat_cols = [c for c in X.columns if X.dtypes[c] == "object"]
num_cols = [c for c in X.columns if X.dtypes[c] != "object"]

# The transformer for the numerical variables
num_preproc = Pipeline([
    ("std", StandardScaler()),
    ("poly", PolynomialFeatures(4))
])

# The transformer for the categorical variables
cat_enc = OneHotEncoder()

# The complete preprocessing step
transf = ... # <----- complete here

# The regression model
reg = Ridge(1e-1)

# The complete pipeline
model = Pipeline([
    ("preproc", transf),
    ("reg", reg)
])

model.fit(Xtrain, ytrain)
ypred = model.predict(Xtest).clip(0, np.inf)

car_pricing.plot_pred_errors(ytest, ypred)
result = permutation_importance(model, Xtest, ytest)
pd.DataFrame({"importance": result["importances_mean"]}, index=Xtest.columns)\
    .sort_values("importance").plot.barh()

```

In [ ]:
from category_encoders import OneHotEncoder
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.inspection import permutation_importance


cat_cols = [c for c in X.columns if X.dtypes[c] == "object"]
num_cols = [c for c in X.columns if X.dtypes[c] != "object"]

# The transformer for the numerical variables
num_preproc = Pipeline([("std", StandardScaler()), ("poly", PolynomialFeatures(4))])

# The transformer for the categorical variables
cat_enc = OneHotEncoder()

# The complete preprocessing step
transf = ColumnTransformer([("num", num_preproc, num_cols), ("cat", cat_enc, cat_cols)])

# The regression model
reg = Ridge(1e-1)

# The complete pipeline
model = Pipeline([("preproc", transf), ("reg", reg)])

model.fit(Xtrain, ytrain)
ypred = model.predict(Xtest).clip(0, np.inf)

car_pricing.plot_pred_errors(ytest, ypred)
result = permutation_importance(model, Xtest, ytest)
_ = (
    pd.DataFrame({"importance": result["importances_mean"]}, index=Xtest.columns)
    .sort_values("importance")
    .plot.barh()
)

**Note**: The `category_encoders.OneHotEncoder` has some parameters for specifying the strategy to apply for **unknown** categories (i.e. categories not seen at training time). Among other possibilities, one can choose between:

* using a **special value** (all zeros)
* raising an error.

One-hot encoding is not appropriate when the number of categories is high in general and especially when there is a large number of rare categories.

### Target Encoding

Target encoding consists in encoding categories as the average of the target value for the subset of samples where that category is observed.

Basically, the encoded value of category $c$, $v_c$ is computed as:

$$v_c = \frac{\sum_{i=1}^N \mathbb I(c_i = c) y_i}{\sum_{i=1}^N \mathbb I(c_i = c)}$$

Where $\mathbb I(.)$ is the indicator function.

To prevent over-fitting categories with low occurrences we can add a smoothing factor $\alpha$:

$$v_c = \frac{\sum_{i=1}^N \mathbb I(c_i = c) y_i + \alpha \bar y}{\sum_{i=1}^N \mathbb I(c_i = c) + \alpha}  $$

where $\bar y$ is the global average of the target values.

To implement this, we can use the `TargetEncoder` transformer from the **`category_encoders`** package. This transformer allows us to further specify:

* the minimum number of occurrences for a category to be considered (otherwise the overall average will be used)
* Unkown values must be treated.

Setting the minimum number of occurrences is a way to deal with **rare categories**. **Unknown categories** can then be handled either by:

* raising an error
* replacing them (like rare categories) with the overall average.

Another common way to manage rare categories is to **group them into a new category** and **assign unknown categories** to the new category.

The main advantage of **target encoding** compared to one-hot encoding is that it **keeps the number of features unchanged**.

In our case, this simplifies the pipeline a lot, and we can use the `PolynomialFeature` transformer on all the features:

In [ ]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from category_encoders import TargetEncoder
from sklearn.linear_model import Ridge, Lasso
from sklearn.pipeline import Pipeline

cat_enc = TargetEncoder(
    cols=cat_cols,
    min_samples_leaf=5,
    smoothing=5.0,
)

preproc = [
    ("categorical", cat_enc),
    ("std", StandardScaler()),
    ("poly", PolynomialFeatures(degree=4, include_bias=False)),
]

reg = Ridge(alpha=1.0e2)

model = Pipeline(preproc + [("reg", reg)])

model.fit(Xtrain, ytrain)
ypred = model.predict(Xtest).clip(0, np.inf)

car_pricing.plot_pred_errors(ytest, ypred)
result = permutation_importance(model, Xtest, ytest)
_ = (
    pd.DataFrame({"importance": result["importances_mean"]}, index=Xtest.columns)
    .sort_values("importance")
    .plot.barh()
)

In this case we can get slightly better results than with one-hot encoding. Probably due to the fact the polynomial expansion acts on both numerical and categorical variables.

As you might guess, ensemble models will probably perform better:

In [ ]:
from category_encoders import TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.pipeline import Pipeline

cat_enc = TargetEncoder(cols=cat_cols, min_samples_leaf=5, smoothing=1.0)

reg = HistGradientBoostingRegressor(
    max_iter=100, learning_rate=0.1, max_depth=7, random_state=1
)

model = Pipeline([("cat_enc", cat_enc), ("reg", reg)])

model.fit(Xtrain, ytrain)
ypred = model.predict(Xtest).clip(0, np.inf)

car_pricing.plot_pred_errors(ytest, ypred)
result = permutation_importance(model, Xtest, ytest)
_ = (
    pd.DataFrame({"importance": result["importances_mean"]}, index=Xtest.columns)
    .sort_values("importance")
    .plot.barh()
)

### CatBoost

As already stated, the Catboost algorithm directly deals with categorical variables by applying its own variant of target encoding:

In [ ]:
from catboost import CatBoostRegressor

reg = CatBoostRegressor(
    cat_features=cat_cols,
    n_estimators=700,
    learning_rate=0.7,
    depth=5,
    random_state=1,
    verbose=False,
)

reg.fit(Xtrain, ytrain)
ypred = reg.predict(Xtest).clip(0, np.inf)

car_pricing.plot_pred_errors(ytest, ypred)
_ = (
    pd.DataFrame({"importance": reg.feature_importances_}, index=Xtest.columns)
    .sort_values("importance")
    .plot.barh()
)

In this case CatBoost is slightly worse than the previous model even though the difference might lie *in the model noise*. The ease of use makes up for this slight performance loss though  (no need to perform any preprocessing).

## Dealing with missing values

So far we have avoided processing missing values by skipping the incomplete rows of the dataset.
Taking those values into account is important since missing values can occur in real life applications.
There are different methods for doing that.

In [ ]:
from sklearn.model_selection import train_test_split

X = df_cars.drop(["price"], axis=1)
y = df_cars.price.values

cat_cols = [c for c in X.columns if X.dtypes[c] == "object"]
num_cols = [c for c in X.columns if X.dtypes[c] != "object"]

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=0)

### Simple imputation

The usual approach to deal with missing values is to **impute** them, which means to replace them with some numerical value computed in a certain way.

The most simple imputation method consists in replacing with an aggregated value computed on all non-missing values.

In [ ]:
from category_encoders import TargetEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

imputer = SimpleImputer(strategy="mean")

cat_enc = TargetEncoder(cols=cat_cols, min_samples_leaf=5, smoothing=10.0)

reg = Ridge(1e1)

model = Pipeline(
    [
        ("cat_enc", cat_enc),
        ("impute", imputer),
        ("std", StandardScaler()),
        ("poly", PolynomialFeatures(3, include_bias=False)),
        ("reg", reg),
    ]
)

model.fit(Xtrain, ytrain)

ypred = model.predict(Xtest).clip(0, np.inf)

car_pricing.plot_pred_errors(ytest, ypred)
result = permutation_importance(model, Xtest, ytest)
_ = (
    pd.DataFrame({"importance": result["importances_mean"]}, index=Xtest.columns)
    .sort_values("importance")
    .plot.barh()
)

### Prediction-based imputation

More advanced methods try to model the relationship between the input features and build a model which predicts the missing value as a function of the other variables.

Since more than one column can have missing values, the prediction is built iteratively. The experimental `IterativeImputer` from Scikit-learn implements such a process:

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from category_encoders import TargetEncoder
from sklearn.impute import IterativeImputer
from sklearn.pipeline import Pipeline

imputer = IterativeImputer(random_state=1)

cat_enc = TargetEncoder(cols=cat_cols, min_samples_leaf=5, smoothing=10.0)

reg = Ridge(1e1)

model = Pipeline(
    [
        ("cat_enc", cat_enc),
        ("impute", imputer),
        ("std", StandardScaler()),
        ("poly", PolynomialFeatures(3, include_bias=False)),
        ("reg", reg),
    ]
)

model.fit(Xtrain, ytrain)

ypred = model.predict(Xtest).clip(0, np.inf)

car_pricing.plot_pred_errors(ytest, ypred)
result = permutation_importance(model, Xtest, ytest)
_ = (
    pd.DataFrame({"importance": result["importances_mean"]}, index=Xtest.columns)
    .sort_values("importance")
    .plot.barh()
)

### A custom imputer*

This section shows how to build our own imputer relying on the Scikit learn API

**Exercise**: Build your own imputer

The `SimpleImputer` seems to work better than the complicated `IterativeImputer`. The former strategy  computes the average or median of the non-missing values for the column. We will try to improve on this simple idea by using a categorical column as a grouping key and compute the average for the category.

So the exercise consists in completing the following code.

**Hints**:

* Assuming the input X data is a pandas dataframe, use the `X.groupby(key)[cols].agg(aggfunc)` construct to compute the aggegation
* Use `np.where` to replace missing values only
* Use the `Series.map()` to apply the computed imputation values based on the grouping key.
* We will not try to deal with rare or unknown categories.

```python
from sklearn.base import BaseEstimator, TransformerMixin


class GroupSimpleImputer(BaseEstimator, TransformerMixin):
    """Compute imputation values based on a grouping key.

    Parameters
    ----------
    group_column: str
        The categorical column used as grouping key
    cols: list of str
        The columns to be imputed

    strategy: str
        The name of the aggregating function to pass to the `agg()` method.
    """
    def __init__(self, group_column=None, cols=None, strategy="mean"):
        self.group_column = group_column
        self.cols = cols
        self.strategy = strategy

    def fit(self, X, y=None):
        """ Fit the estimator

        Parameters
        ----------
        X: pandas.DataFrame
            The input dataframe
        y: array_like
            The target value (unused here)

        Returns
        -------
        self: the fitted model
        """
        self.group_means_ = ...
        return self

    def transform(self, X):
        """ Impute missing values

        Parameters
        ----------
        X: pandas.DataFrame
            The input dataframe

        Returns
        -------
        X_imputed: pandas.DataFrame
            A copy of the input DataFrame with the missing values imputed.
        """
        Xtr = X.copy()
        ...
        return Xtr
```

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin


class GroupSimpleImputer(BaseEstimator, TransformerMixin):
    """Compute Imputation values based on a grouping key

    Parameters
    ----------
    group_column: str
        The categorical column used as grouping key
    cols: list of str
        The columns to be imputed

    strategy: str
        The name of the aggregating function to pass to the `agg()` method.
    """

    def __init__(self, group_column=None, cols=None, strategy="mean"):
        self.group_column = group_column
        self.cols = cols
        self.strategy = strategy

    def fit(self, X, y=None):
        """Fit the estimator

        Parameters
        ----------
        X: pandas.DataFrame
            The input dataframe
        y: array_like
            The target value (unused here)

        Returns
        -------
        self: the fitted model
        """
        self.group_means_ = X.groupby(self.group_column)[self.cols].agg(self.strategy)
        return self

    def transform(self, X):
        """Impute missing values

        Parameters
        ----------
        X: pandas.DataFrame
            The input dataframe

        Returns
        -------
        X_imputed: pandas.DataFrame
            A copy of the input DataFrame with the missing values imputed.
        """
        Xtr = X.copy()
        for c in self.cols:
            group_mean = Xtr[self.group_column].map(self.group_means_[c])
            Xtr[c] = np.where(Xtr[c].isnull(), group_mean, Xtr[c])
        return Xtr

We can now include it in the same pipeline as above:

In [ ]:
imputer = GroupSimpleImputer("brand", ["tax", "mpg"])
cat_enc = TargetEncoder(cols=cat_cols, min_samples_leaf=10, smoothing=5.0)

reg = Ridge(1e1)

model = Pipeline(
    [
        ("impute", imputer),
        ("cat_enc", cat_enc),
        ("std", StandardScaler()),
        ("poly", PolynomialFeatures(3, include_bias=False)),
        ("reg", reg),
    ]
)

model.fit(Xtrain, ytrain)

ypred = model.predict(Xtest).clip(0, np.inf)

car_pricing.plot_pred_errors(ytest, ypred)
result = permutation_importance(model, Xtest, ytest)
_ = (
    pd.DataFrame({"importance": result["importances_mean"]}, index=Xtest.columns)
    .sort_values("importance")
    .plot.barh()
)

### Models that natively handle missing values

Many ensemble models can natively deal with missing values:

* `HistGradientBoostingRegressor`
* `XGBoostRegressor`
* `CatBoostRegressor`

Note that `CatBoost` natively supports both categorical features and missing values. That makes it a good candidate for quick investigations (on feature importances for instance).

In [ ]:
model = CatBoostRegressor(
    cat_features=cat_cols,
    n_estimators=100,
    random_state=1,
    verbose=False,
)

model.fit(Xtrain, ytrain)
ypred = model.predict(Xtest).clip(0, np.inf)

car_pricing.plot_pred_errors(ytest, ypred)
result = permutation_importance(model, Xtest, ytest)
_ = (
    pd.DataFrame({"importance": result["importances_mean"]}, index=Xtest.columns)
    .sort_values("importance")
    .plot.barh()
)

Here we did not try to have the best model but simply to see if there are useless variables. Based on the feature importances, we can decide whether to keep or not some features.

Here the missing value indicator seems to be useless for instance.

## Optimizing the hyper-parameters


### Proper practices for model (and hyper-parameters) selection

An estimator's hyper-parameters are those that need to be set before fitting the model. That is why they are given to the constructor.

In the above examples, we modified those parameters manually in order to have the best performance on the test set. While simple and effective, this is not considered a good practice. Indeed by doing so, we implicitly overfit those hyper-parameters for the test-set, which biases our performance estimation.

A proper procedure would involve **excluding the test-set from the hyper-parameters optimization** (or more generally, from the **model selection**).

The first step would then involve partitioning the full dataset into **development and test sets** (note the slight change of vocabulary in order to make things less ambiguous).

Secondly, an hyper-parameter selection strategy is based on two components:

* An hyper-parameter search strategy: which hyper-parameters combinations will be tested ?
* A performance evaluation strategy: how do I estimate the performance of a given model candidate ?

A typical hyper-parameter selection process will look like the following:

1. For each hyper-parameter combination (selected using the search strategy)
    1. Instanciate a model with these hyper-parameters
    2. Fit the model
    2. Emit predictions
    2. Evaluate the model performance
2. Select the hyper parameter combination leading to the best performance
4. Retrain a model with those hyper-parameters on the full dev-set
5. Estimate the final performance on the test set.

### Performance evaluation strategies

We describe here performance evaluation strategies.
Note that while the main intent here is to describe how to make the best use of the development set for training and evaluating the candidate models, the same strategies can actually be applied for the initial partitioning of the full dataset (which is used for the performance evaluation of the final model). Let's concentrate for now on the first case.


#### Train /  validation

The simplest way to evaluate the performance of a model instance will be to perform the training / validation split strategy on the development set.

For instance with this dataset made of 9 samples:
<span style="border: 1px solid black"> 1, 2, 3, 4, 5, 6, 7, 8, 9 </span>

<!--
The dataset is split in 3: <span style="border: 1px solid black"> 1, 2, 3 </span>,
<span style="border: 1px solid black"> 4, 5, 6 </span>,
<span style="border: 1px solid black"> 7, 8, 9 </span>
-->

Using the color convention:
<span style="border: 1px solid black; background-color: lime"> training </span> and
<span style="border: 1px solid black; background-color: cyan"> validation </span>

We could split this way:
<span style="border: 1px solid black; background-color: lime"> 1, 2, 3, 4, 5, 6 </span>,
<span style="border: 1px solid black; background-color: cyan"> 7, 8, 9 </span>

It is common to **shuffle** the samples before splitting.


The evaluation strategy will consist in:

1. Training a model instance on the (sub-)train set
2. Predict the target values for the validation set
3. Evaluate the performance based on the predictions and ground truth values on the validation set.

The main drawback of this approach is that it uses only a small subset of the development set to estimate the performance. And not all data points are used for training.

Its main advantage is that for $P$ hyper-parameters combinations tested, it requires only $P$ model fits.

A trade-off needs to be found for the validation set size, since:

- too small a validation set will lead to unreliable performance measures;
- too large a validation set will lead to a smaller training set, degrading the predictive performance.


Since this is the strategy we have used to evaluate the final model performance, if we use this strategy to select the hyper-parameter set leading to the best performance, it leads to the **train / validation / test strategy** where in the end we have 3 data subsets:

* The training set used to train the models
* The validation set used to estimate the model performances and thus select the best parameters
* The test set used to assess the final performance.

For instance:
<span style="border: 1px solid black; background-color: lime; padding: 3px">
    <span style="border: 1px solid black; background-color: lime; margin: 1px">
        1, 2, 3, 4
    </span>
    <span style="border: 1px solid black; background-color: cyan; margin: 1px">
        5, 6
    </span>
</span>,
<span style="border: 1px solid black; background-color: cyan"> 7, 8, 9 </span>

#### K-Fold cross-validation

K-Fold cross validation allows to use the full development dataset set to evaluate the performance of a model condidate. It works by splitting the data set into $K$ subsets, and using each one iteratively for validation while fitting the model on the union of the remaining subsets.

It is performed this way:

1. For $i$ in 1 to $K$:

    1. keep subset $i$ as the validation set
    2. train the model on the union of the remaining subsets
    3. emit predictions for the subset $i$ (current validation set)

2. Compute the performance using all predicted values


Iteration 1:
<span style="border: 1px solid black; background-color: cyan"> 1, 2, 3 </span>,
<span style="border: 1px solid black; background-color: lime"> 4, 5, 6 </span>,
<span style="border: 1px solid black; background-color: lime"> 7, 8, 9 </span>

Iteration 2:
<span style="border: 1px solid black; background-color: lime"> 1, 2, 3 </span>,
<span style="border: 1px solid black; background-color: cyan"> 4, 5, 6 </span>,
<span style="border: 1px solid black; background-color: lime"> 7, 8, 9 </span>

Iteration 3:
<span style="border: 1px solid black; background-color: lime"> 1, 2, 3 </span>,
<span style="border: 1px solid black; background-color: lime"> 4, 5, 6 </span>,
<span style="border: 1px solid black; background-color: cyan"> 7, 8, 9 </span>


This way we have performed predictions for all the data points without using the same points to train and to predict.

If we use this strategy to estimate the performance of a model family with $P$ hyper-parameter combinations, with $K$ subsets, we need to train $K \times P$ models.

Moreover, since the model is trained with a fraction $1 - \frac{1}{K}$ of the full train set, small values of $K$ may lead to results which will be very different from those obtained with the model trained on the full train set.

Finally if we used this strategy for **both hyper-parameter selection and final performance evaluation**, we end-up with **2 nested cross-validation loops** which can lead a very large number of trained models.

#### Leave-one-out strategy

The leave-one-out strategy is an extreme version of the K-Fold strategy where $K$ is equal to the number $N$ of data points in the development set.

Iteration 1:
<span style="border: 1px solid black; background-color: cyan"> 1</span>,
<span style="border: 1px solid black; background-color: lime"> 2</span>,
<span style="border: 1px solid black; background-color: lime"> 3</span>,
<span style="border: 1px solid black; background-color: lime"> 4</span>,
<span style="border: 1px solid black; background-color: lime"> 5</span>,
<span style="border: 1px solid black; background-color: lime"> 6</span>,
<span style="border: 1px solid black; background-color: lime"> 7</span>,
<span style="border: 1px solid black; background-color: lime"> 8</span>,
<span style="border: 1px solid black; background-color: lime"> 9</span>

Iteration 2:
<span style="border: 1px solid black; background-color: lime"> 1</span>,
<span style="border: 1px solid black; background-color: cyan"> 2</span>,
<span style="border: 1px solid black; background-color: lime"> 3</span>,
<span style="border: 1px solid black; background-color: lime"> 4</span>,
<span style="border: 1px solid black; background-color: lime"> 5</span>,
<span style="border: 1px solid black; background-color: lime"> 6</span>,
<span style="border: 1px solid black; background-color: lime"> 7</span>,
<span style="border: 1px solid black; background-color: lime"> 8</span>,
<span style="border: 1px solid black; background-color: lime"> 9</span>

Iteration 3:
<span style="border: 1px solid black; background-color: lime"> 1</span>,
<span style="border: 1px solid black; background-color: lime"> 2</span>,
<span style="border: 1px solid black; background-color: cyan"> 3</span>,
<span style="border: 1px solid black; background-color: lime"> 4</span>,
<span style="border: 1px solid black; background-color: lime"> 5</span>,
<span style="border: 1px solid black; background-color: lime"> 6</span>,
<span style="border: 1px solid black; background-color: lime"> 7</span>,
<span style="border: 1px solid black; background-color: lime"> 8</span>,
<span style="border: 1px solid black; background-color: lime"> 9</span>

...

Iteration 9:
<span style="border: 1px solid black; background-color: lime"> 1</span>,
<span style="border: 1px solid black; background-color: lime"> 2</span>,
<span style="border: 1px solid black; background-color: lime"> 3</span>,
<span style="border: 1px solid black; background-color: lime"> 4</span>,
<span style="border: 1px solid black; background-color: lime"> 5</span>,
<span style="border: 1px solid black; background-color: lime"> 6</span>,
<span style="border: 1px solid black; background-color: lime"> 7</span>,
<span style="border: 1px solid black; background-color: lime"> 8</span>,
<span style="border: 1px solid black; background-color: cyan"> 9</span>


It has the advantage to perform predictions closer to those of the model trained on the full dataset but has the main disadvantage to require $N \times P$ model estimations.

As a consequence it will usually be used when the number of samples is small. However, if the model is very sensitive to some of the training points, it might lead to poor performance evaluation without no way detect it.

As a comparison, in the $K$-fold strategy, performances can be computed for each subset and an instability can be detected as a highly variable performance measure across the subsets.

#### Shuffle split

Shuffle split performs the train / validation strategy repeatedly while the data points are shuffled each time. We can use it to get an idea of the performance measure distribution.

Iteration 1:
<span style="border: 1px solid black; background-color: lime"> 1</span>,
<span style="border: 1px solid black; background-color: cyan"> 2</span>,
<span style="border: 1px solid black; background-color: lime"> 3</span>,
<span style="border: 1px solid black; background-color: lime"> 4</span>,
<span style="border: 1px solid black; background-color: lime"> 5</span>,
<span style="border: 1px solid black; background-color: lime"> 6</span>,
<span style="border: 1px solid black; background-color: cyan"> 7</span>,
<span style="border: 1px solid black; background-color: cyan"> 8</span>,
<span style="border: 1px solid black; background-color: lime"> 9</span>


Iteration 2:
<span style="border: 1px solid black; background-color: lime"> 1</span>,
<span style="border: 1px solid black; background-color: lime"> 2</span>,
<span style="border: 1px solid black; background-color: lime"> 3</span>,
<span style="border: 1px solid black; background-color: cyan"> 4</span>,
<span style="border: 1px solid black; background-color: cyan"> 5</span>,
<span style="border: 1px solid black; background-color: lime"> 6</span>,
<span style="border: 1px solid black; background-color: cyan"> 7</span>,
<span style="border: 1px solid black; background-color: lime"> 8</span>,
<span style="border: 1px solid black; background-color: lime"> 9</span>

...

Iteration T:
<span style="border: 1px solid black; background-color: cyan"> 1</span>,
<span style="border: 1px solid black; background-color: lime"> 2</span>,
<span style="border: 1px solid black; background-color: cyan"> 3</span>,
<span style="border: 1px solid black; background-color: lime"> 4</span>,
<span style="border: 1px solid black; background-color: lime"> 5</span>,
<span style="border: 1px solid black; background-color: lime"> 6</span>,
<span style="border: 1px solid black; background-color: lime"> 7</span>,
<span style="border: 1px solid black; background-color: cyan"> 8</span>,
<span style="border: 1px solid black; background-color: lime"> 9</span>

#### Scikit-learn implementations

Scikit-learn provides means of performing those different performance evaluation strategies.

Let's rebuild our data set:

In [ ]:
from sklearn.model_selection import train_test_split

exclude_columns = ["price", "tax_mpg_missing"]

X = df_cars.drop(exclude_columns, axis=1)
y = df_cars.price.values

Xdev, Xtest, ydev, ytest = train_test_split(X, y, random_state=0)

And let's start from an initial pipeline (here just a single estimator):

In [ ]:
model = CatBoostRegressor(
    cat_features=cat_cols, n_estimators=100, random_state=1, verbose=False
)

To estimate a cross-validation score, Scikit-learn provides the `sklearn.metrics.cross_val_score` function. This function takes as arguments:

* An estimator
* The input features
* The target value
* A cross validation object
* A scoring function

The cross-validator object must be :
* an iterable object providing a sequence of `(train_indexes, valid_indexes)`.
* an object having a `.split(X, y=None)` method returning the same type of sequence

Here we use the functionalities of the `sklearn.model_selection` subpacakge to implement **shuffle split**. We show that the sets of indices are disjoined and cover the complete dataset:

In [ ]:
from sklearn.model_selection import ShuffleSplit

print(f"Number of dev samples: {len(Xdev)}")
cv_shuffle_split = ShuffleSplit(n_splits=5, test_size=0.3, random_state=1)
for itrain, itest in cv_shuffle_split.split(Xdev):
    print(itrain, itest, len(set(itrain) & set(itest)), len(set(itrain) | set(itest)))

And we can then use it with the `cross_val_score` function. For each split, the function fits a new model on the train samples and computes the score on the validation set:

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, Xdev, ydev, scoring="r2", cv=cv_shuffle_split)
scores

We retrieve the scores computed on each split.

This strategy can be used to estimated that variation of score induced by the split only. Here, we have a spread:

In [ ]:
print(scores.std())

This gives us an indication that performance measures differences less than say 0.003 are probably not really significant.

Note that the **train / validation** split strategy can be understood as a suffle split with only one split:

In [ ]:
cv_train_valid = ShuffleSplit(n_splits=1, test_size=0.3, random_state=1)
for itrain, itest in cv_train_valid.split(Xdev):
    print(itrain, itest, len(set(itrain) & set(itest)), len(set(itrain) | set(itest)))
scores = cross_val_score(model, Xdev, ydev, scoring="r2", cv=cv_train_valid)
scores

In the above code we indicated that we use the `"r2"` score. This score is automatically computed using the prediction of the model. Since the beginning we have computed the score using the clipped predictions. If we want to compute the score the same way, we need to pass a **custom scorer**.

A **scorer** is a callable object with signature `scorer(model, X, y)`. Scikit-learn makes it easy to create a scorer object from a simpler score function using `sklearn.metrics.make_scorer()`:

In [ ]:
from sklearn.metrics import r2_score, make_scorer


def r2_score_clipped(ytrue, ypred):
    return r2_score(ytrue, ypred.clip(0, np.inf))


r2_scorer = make_scorer(r2_score_clipped)

scores = cross_val_score(model, Xdev, ydev, scoring=r2_scorer, cv=cv_train_valid)
scores

The **K-Fold strategy** can be implemented using the `sklearn.model_selection.KFold` object:

In [ ]:
from sklearn.model_selection import KFold

cv_kfold = KFold(3, shuffle=True, random_state=1)

for itrain, ivalid in cv_kfold.split(Xdev):
    print(itrain, ivalid)

To use this partioning strategy, we pass the `cv_kfold` object as the `cv` parameter of the `cross_val_score`:

In [ ]:
scores = cross_val_score(model, Xdev, ydev, scoring=r2_scorer, cv=cv_kfold)
scores

For cross validation strategies that perform a **permutation of the input set** ,i.e. for each sample is used once and only once in the validation set, it is possible to retrieve the predictions for each input point using `cross_val_predict`:

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import r2_score

ypred = cross_val_predict(model, Xdev, ydev, cv=cv_kfold).clip(0, np.inf)
car_pricing.plot_pred_errors(ydev, ypred)

This has the advantage to let us modify the raw prediction of the model before computing the score ourself.

This approach would not work with the train / validation or the shuffle split strategy since they do not allow to get one prediction by sample.

The **leave-one-out strategy** is implemented by the `sklearn.model_selection.LeaveOneOut` object but it would be too costly to perform here.

### Search strategies

#### Grid Search

The Grid search strategy consists in proposing a set of values to be tested for each hyper-parameter of interest. We then simply iterate through all combinations of hyper parameter values (Cartesian product). This is a brute-force approach.

It has the advantage to scan all the proposed values, but the number of tested combinations quicky explodes: it increases both with the number of parameters to scan, and the number of values to test for each of them.

If there are $p$ parameters with $n_p$ values to test for each one, the number $P$ of combinations is:

$$ P = \prod_{i=1}^p n_i $$

Imagine we have first tested some combinations manually and found a reasonably good solution as here:

In [ ]:
from catboost import CatBoostRegressor

model = CatBoostRegressor(
    n_estimators=200,
    depth=7,
    learning_rate=0.5,
    cat_features=cat_cols,
    verbose=False,
    random_seed=1,
)

model.fit(Xdev, ydev)

ypred = model.predict(Xtest).clip(0, np.inf)
car_pricing.plot_pred_errors(ytest, ypred)

We can then use the `sklearn.model_selection.GridSearchCV` to perform a grid search. The parameter grid is given as a dictionary with the parameter name as key and the list of values to scan as values:

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold

model = CatBoostRegressor(cat_features=cat_cols, verbose=False, random_seed=1)

param_grid = {"n_estimators": [150, 200, 300], "depth": [5, 7], "learning_rate": [0.5]}

cv = KFold(3, shuffle=True, random_state=1)
cv_model = GridSearchCV(
    model, param_grid=param_grid, cv=cv, n_jobs=2, scoring=r2_scorer, verbose=2
)
cv_model.fit(Xdev, ydev)

So we have:

* 3 values for `n_estimators`
* 2 values for `depth`
* 1 value for `learning_rate`

Which leads to $3 \times 2 \times 1 = 6$ parameter combinations. Since we perform a 3-fold cross-validation, this will lead to $6 \times 3 = 18$ model fits.

Here we have given the `n_jobs=2` to use 2 CPUs in parallel, we could have written used the value -1 to use all.

Once the model is trained, we have access to the best parameter combination:

In [ ]:
cv_model.best_params_, cv_model.best_score_

Some detailed information is available through the `cv_results_` attribute. For instance we can have the mean score for each parameter set:

In [ ]:
cv_model.cv_results_.keys()

In [ ]:
cv_model.cv_results_["mean_test_score"]

These scores correspond to the following parameter sets:

In [ ]:
for pset, mean_score in zip(
    cv_model.cv_results_["params"], cv_model.cv_results_["mean_test_score"]
):
    print(f"{pset}: {mean_score}")

We can indeed check that the best combination corresponds to the highest score.

A model using the full dev set has been automatically refit with the best found parameters, now we can evaluate the final model on the test set:

In [ ]:
ypred = cv_model.predict(Xtest).clip(0, np.inf)
car_pricing.plot_pred_errors(ytest, ypred)

And we can see that the scores have slightly improved compared to our original selection.

#### Randomized search

Grid search is not adapted when the number of parameters to scan is high. If that's the case, instead of scanning all possible combinations, we can randomly sample a subset of combinations. We will set by advance the number of combinations to test, allowing to perform the selection in a more controlled time.

The distributions of parameters to use can be given as:

* a sequence of values which will be picked up with equal probability
* a `scipy.stats` distribution.



In [ ]:
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.utils import check_random_state
from scipy.stats import uniform

random_state = check_random_state(0)

model = CatBoostRegressor(cat_features=cat_cols, verbose=False, random_seed=1)

param_distr = {
    "n_estimators": np.arange(100, 500),  # uniformly picked between 100 and 499
    "depth": np.arange(3, 11),  # uniformly picked between 3 and 10
    "learning_rate": uniform(0, 1),  # Uniformly drawn in the [0 1] interval
}

cv = KFold(3, shuffle=True, random_state=random_state)
cv_model = RandomizedSearchCV(
    model,
    n_iter=5,
    param_distributions=param_distr,
    random_state=random_state,
    scoring=r2_scorer,
    cv=cv,
    n_jobs=2,
    verbose=2,
)
_ = cv_model.fit(Xdev, ydev)

Note that we use arrays for uniformly choosing between discrete values, and distributions for continuous values. The use of discrete distributions like Poisson or Negative Binomial would be possible depending on the desired behavior.

Again we scan the different values in parallel.

This strategy allows to deal with a large number of parameters, but it is of course not guaranteed to find the actual optimum. On the other hand, it can reach parameter values that would be impractical to test with a grid search, because its set of values to test is fixed.

When the search is finished we can retrieve the best parameters and the best score the same way as with grid search:

In [ ]:
cv_model.best_params_, cv_model.best_score_

And finally check the result on the test set:

In [ ]:
ypred = cv_model.predict(Xtest).clip(0, np.inf)
car_pricing.plot_pred_errors(ytest, ypred)

#### Pipeline hyper-parameters

We have seen that parameter grids or parameter distributions are passed to the Search objects using dictionaries with the parameter names as keys.

While the parameter name is obvious for single estimators as in the previous example, it becomes more complicated with a composed model as below:

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.experimental import enable_iterative_imputer
from sklearn.pipeline import Pipeline
from sklearn.impute import IterativeImputer
from category_encoders import TargetEncoder

cat_enc = TargetEncoder(cols=cat_cols, min_samples_leaf=100)

reg = HistGradientBoostingRegressor(
    max_iter=400, min_samples_leaf=10, l2_regularization=0.3
)

model = Pipeline([("categories", cat_enc), ("reg", reg)])

model.fit(Xdev, ydev)

ypred = model.predict(Xtest).clip(0, np.inf)
car_pricing.plot_pred_errors(ytest, ypred)

Here we have a 2-steps pipeline. And note that some parameters in the two steps have the same names. To be able to address unambiguously each parameter, Scikit-learn uses the following notations:

* `categories__min_samples_leaf`
* `reg__max_iter`
* `reg__min_samples_leaf`
* `reg__l2_regularization`

We see that the parameters are addressed by prefixing each estimator name by the step name followed by `'__'`. This is the reason why all composed estimators have to name their components.

This way several levels of imbrication are possible.

**Exercise**:

Considering the following composed estimator:

```python

num_transf = Pipeline([
    ("std", StandardScaler()),
    ("pca", PCA(n_components=0.9)),
])

transf = ColumnTransformer([
    ("numeric", num_transf, num_cols),
    ("categ", OneHotEncoder(), cat_cols)
])

model = Pipeline([
    ("transf", transf),
    ("reg", XGBoostRegressor(n_estimators=100))
])

```

How would you address:

* The `n_components` parameter of the `PCA` transformer?
* The `min_samples_leaf` parameter of the `OneHotEncoder` transformer?
* The `n_estimators` parameter of the `XGBoostRegressor` regressor?


* `transf__numeric__pca__n_components`
* `transf__categ__min_samples_leaf`
* `reg__n_estimators`

**Exercise**: Starting from the model manually found above (the one composed of a TargetEncoder and an HistGradientBoostingRegressor estimator):

1. Perform a randomized search for the 4 parameters used, using the following distributions:

    * `TargetEncoder.min_samples_leaf`: uniform between 1 and 20
    * `HistGradientBoostingRegressor.max_iter`: uniform between 100 and 400
    * `HistGradientBoostingRegressor.min_samples_leaf`: uniform between 1 and 20
    * `HistGradientBoostingRegressor.l2_regularization`: exponential distribution with 0.1 mean (`scipy.stats.expon(scale=0.1)`)
3. Use 20 iterations with a 3-fold cross validation

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import HistGradientBoostingRegressor
from category_encoders import TargetEncoder
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import expon
from sklearn.utils import check_random_state
from mll1 import car_pricing

model = Pipeline(
    [
        ("categories", TargetEncoder(cols=cat_cols)),
        ("reg", HistGradientBoostingRegressor()),
    ]
)

param_distr = {
    "categories__min_samples_leaf": np.arange(1, 21),
    "reg__max_iter": np.arange(100, 401),
    "reg__min_samples_leaf": np.arange(1, 21),
    "reg__l2_regularization": expon(scale=0.1),
}

rng = check_random_state(1)

cv = KFold(3, shuffle=True, random_state=rng)

# Use the score function from
# the module instead of the one declared in the notebook
# to ensure it will be available in other notebooks / programs
r2_scorer = make_scorer(car_pricing.r2_score_clipped)


cv_model = RandomizedSearchCV(
    model,
    n_iter=20,
    cv=cv,
    param_distributions=param_distr,
    random_state=rng,
    n_jobs=2,
    scoring=r2_scorer,
    verbose=True,
)

_ = cv_model.fit(Xdev, ydev)

Note that we use a score function defined in the `car_pricing` module, so that the function is always available when the model is reloaded from another notebook / application.

In [ ]:
cv_model.best_params_, cv_model.best_score_

In [ ]:
ypred = cv_model.predict(Xtest).clip(0, np.inf)
car_pricing.plot_pred_errors(ytest, ypred)
result = permutation_importance(cv_model, Xtest, ytest)
_ = (
    pd.DataFrame({"importance": result["importances_mean"]}, index=Xtest.columns)
    .sort_values("importance")
    .plot.barh()
)

## Model persistence

Scikit-learn models natively support serialization operations (that is, saving objects to byte strings or files, and loading objects from those strings).
This can be done by using the standard `pickle` library.

However, since those models may contain large arrays, for performance and space-efficiency it is recommended to use the `joblib` library instead. The `joblib` library is used by Scikit-learn to perform parallel tasks. This library proposes some optmized functions to serialize and de-serialize objects (which are needed to exchange data through parallel processes, but can also be used for persistence):

In [ ]:
import joblib

joblib.dump(cv_model, "./cars_price_model.pkl")

The model can then be reloaded and used:

In [ ]:
loaded_model = joblib.load("./cars_price_model.pkl")
ypred = loaded_model.predict(Xtest).clip(0, np.inf)

## Deployment

Now that we have a predictive model ready, we will focus on making it available by the final user.
This means we need to build a user interface, so that the user can submit information and retrieve the response.

There are many modalities to deploy a predictive model depending on the type of interaction expected:

* As a command line program: can be used by humans and other programs
* As a REST API: can be used by other programs
* As a software plug-in: to be integrated in another program
* As a Web or Desktop application: to be used interactively by humans.

Note that some mixed deployments are also possible. For instance you can deploy the model as a REST API and build other modalities (command line, plug-in, application, ...) on top of it.

### Voilà

In the notebook 4, we show an example of a very simple web application written directly as a Jupyter notebook and exposed using `Voilà`.

The interactive part of the application is written using `ipywidgets`.

Note that the voilà application can be launched using the button in the Jupyter toolbar (mainly for tests) or using a command line.

The following cell saves some information that will be used to build this application.

In [ ]:
import json

model_per_brand_list = (
    X.groupby("brand")["model"].agg(lambda x: sorted(set(x))).to_dict()
)

features_per_model_lists = (
    X.groupby("model")[["engineSize", "transmission", "fuelType"]]
    .agg(lambda x: sorted(set(x)))
    .to_dict("index")
)

config_dict = {
    "model_per_brand": model_per_brand_list,
    "features_per_models": features_per_model_lists,
    "columns": list(X.columns),
}

with open("./config.json", "w") as outfile:
    json.dump(config_dict, outfile)

# config_dict

### Streamlit

Streamlit is a framework allowing to easily build applications. It is aimed at data scientists to easier the deployment of demonstration applications using machine learning models.

You will find an example of application in the `teacher_version/python/car_pricing_app.py` file.

One of the main features of `streamlit` is that interactive applications can be written in a very straghtforward way
without having to take care of event handling between widgets.

This is an excerpt of the application code. We can see that we first retrieve the brand using a streamlit (the `st` module) select box (a dropdown menu).

The we use the retrieved value to configure the possible choices for the model dialog.

```python
brand = st.selectbox(
    label="Brand:",
    options=sorted(config["model_per_brand"].keys())
)

model = st.selectbox(
    label="Model:",
    options=sorted(config["model_per_brand"][brand]) + ["Other"]
)
```

In the voilà application using `ipywidgets` we need to explicitly update the choices of the model selector when
the brand selection changes.

```python
brands = w.Dropdown(options=sorted(config["model_per_brand"].keys()),
                    description="Brand: ", value=None)
models = w.Dropdown(description="Model: ", disabled=True)

def update_models(*args):
    # If a brand is selected
    # 1. Enable the models selector
    # 2. Set the list corresponding to the brand
    if brands.value is not None:
        models.disabled = False
        models.options = sorted(config["model_per_brand"][brands.value]) + ["Other"]
    else:  # Otherwise disable the models selector
        models.disabled = True

brands.observe(update_models)
```


Streamlit performs this magic by rerunning the application in a loop. As a consequence, the code is re-excuted several times. To prevent it, it is possible to wrap time consuming steps which need to be done only onces, like loading the model, into functions decorated with the `st.cache` decorator :


```python
@st.cache(allow_output_mutation=True)
def load_model_and_config():
    reg = joblib.load(model_dir / "cars_price_model.pkl")
    with open(model_dir / "config.json") as infile:
        config = json.load(infile)
    return reg, config
```

The `allow_output_mutation=True` is necesseray because, by default, Streamlit checks that no modifications are done on the cached outputs. Since we call methods on the loaded model, Streamlit might assume the object can be modified.

### FastAPI

Building a REST API with FastAPI can be very fast (indeed). The code below gives a quick and dirty example of such an implemetation :

```python
import fastapi
import joblib
import pandas as pd
import json
import numpy as np
import pathlib

# Load the model and config information
model_dir = pathlib.Path("../notebooks")
model = joblib.load(model_dir / "cars_price_model.pkl")
with open(model_dir / "config.json") as infile:
    config = json.load(infile)


# Build the API
app = fastapi.FastAPI()


@app.get("/price/")  # Access through GET HTTP requests at the "<server_address>/price/" url.
def price(brand: str, car_model: str,
          engine_size: float, fuel_type: str, transmission: str,
          mileage: float, year: int):
    df = pd.DataFrame([{
        "brand": brand,
        "model": car_model,
        "engineSize": engine_size,
        "fuelType": fuel_type,
        "transmission": transmission,
        "year": year,
        "mileage": mileage,
        "tax": np.nan,
        "mpg": np.nan
    }])[config["columns"]]
    return {"price": model.predict(df)[0]}
```

Of course more work would be needed to make it user proof. provide this code in the `teacher_version/python/car_pricing_api.py` file.

The server could be launch for instance from the `python` directory using :

```bash
uvicorn car_pricing_api:app
```

FastAPI builds an automatic documentation page available at `<server_address>/docs/` which allows to send requests to the server.

For instance, one could send a HTTP GET query using :

`http://<server_address>/price/?brand=Audi&car_model=A1&engine_size=1.0&fuel_type=Diesel&transmission=Automatic&mileage=100000&year=2003`

## Going further

### Derivation of the least squares solution

We define the gradient of the function $L(\boldsymbol w, b)$ with respect to $\boldsymbol w$  using the directional derivative in direction $\boldsymbol z$ as the quantity noted $\frac{\partial L(\boldsymbol w, b)}{\partial \boldsymbol w}$ such that:

$$\lim_{h \rightarrow 0} \frac{L(\boldsymbol w + h\boldsymbol z, b) - L(\boldsymbol w, b)}{h}
=\frac{\partial L(\boldsymbol w, b)}{\partial \boldsymbol w}^T \boldsymbol z$$

We have

$$L(\boldsymbol w, b) = \|X\boldsymbol w - b \boldsymbol 1 - \boldsymbol y\|^2 = (X\boldsymbol w - b \boldsymbol 1- \boldsymbol y)^T(X\boldsymbol w - b \boldsymbol 1 - \boldsymbol y)$$

And

\begin{eqnarray}
L(\boldsymbol w + h\boldsymbol z, b)
&=& (X(\boldsymbol w + h\boldsymbol z) - b \boldsymbol 1 - \boldsymbol y)^T(X(\boldsymbol w + h\boldsymbol z) - b \boldsymbol 1 - \boldsymbol y) \\
&=& ((X\boldsymbol w - b \boldsymbol 1 - \boldsymbol y) +  hX\boldsymbol z)^T ((X\boldsymbol w - b \boldsymbol 1 - \boldsymbol y) +  hX\boldsymbol z)\\
&=& L(\boldsymbol w, b) + 2 h(X\boldsymbol w - b \boldsymbol 1 - \boldsymbol y)^TX \boldsymbol z + h^2\boldsymbol z^T X^T X \boldsymbol z
\end{eqnarray}

So we get:



$$\lim_{h \rightarrow 0} \frac{L(\boldsymbol w + h\boldsymbol z, b) - L(\boldsymbol w, b)}{h}
= 2 (X\boldsymbol w - b \boldsymbol 1 - \boldsymbol y)^TX \boldsymbol z$$

which lead to:

$$\frac{\partial L}{\partial \boldsymbol w} = 2 X^T(X \boldsymbol w + b\boldsymbol 1 - \boldsymbol y)$$

The analogue for $b$ is obtained using the usual derivative:

$$\frac{\partial L}{\partial b} = 2 \boldsymbol 1_N^T(X \boldsymbol w + b\boldsymbol 1 - \boldsymbol y)$$

Imposing that both values are 0 we get:

$$\frac{\partial L}{\partial \boldsymbol w} = 2 X^T(X \boldsymbol w + b\boldsymbol 1 - \boldsymbol y) = \boldsymbol 0$$
$$\frac{\partial L}{\partial b} = 2 \boldsymbol 1_N^T(X \boldsymbol w + b\boldsymbol 1 - \boldsymbol y) = 0$$

The optimal value of $b$ is then:
$$b = \frac{1}{N}\boldsymbol 1_N^T(\boldsymbol y - X \boldsymbol w)$$

We substitute it in the first equation and get:

$$X^T(X \boldsymbol w + \boldsymbol 1_N(\frac{1}{N}\boldsymbol 1_N^T(\boldsymbol y - X \boldsymbol w) - \boldsymbol y) = \boldsymbol 0$$

Let $I_N$ be the $N\times N$ identity matrix and $C_N = I_N - \frac{1}{N}\boldsymbol 1_N \boldsymbol 1_N^T$ the $N \times N$ centering matrix, we get:

$$X^TC_N(X \boldsymbol w - \boldsymbol y) = 0$$

$$\tilde{\boldsymbol w} = (X^T C_N X)^{-1}X^T C_N \boldsymbol{ y}$$

Noticing that $C_NC_N = C_N$ (idem potent) and letting $\hat X = C_N X$ and $\boldsymbol{\hat y} = C_N \boldsymbol y$ be the centered version of $X$ and $\boldsymbol y$ respectively, we get:

$$\tilde{\boldsymbol w} = (\hat X^T \hat X)^{-1}\hat X^T \boldsymbol{\hat y}$$

$$b = \bar y - \boldsymbol{\bar x^T} \tilde{\boldsymbol w}$$

Where $\bar y$ is the average target and $\bar{\boldsymbol x}$ is the barycenter of the point cloud.

### More model families

#### Kernel models

Under some conditions, the weight vectors of the linear models can be expressed as a linear combination of the training data points ([**representer theorem**](https://en.wikipedia.org/wiki/Representer_theorem)). This means that the model output can be expressed as combinations of dot products between new data points and the training data points only. The [**kernel trick**](https://en.wikipedia.org/wiki/Kernel_metho) consists in replacing the linear dot product by any function, a.k.a kernel, which have the properties of a dot product.

$$ f(\boldsymbol x) =
   \boldsymbol w^T \boldsymbol x + b \rightarrow
   \sum_{i=1}^N \alpha_i  \boldsymbol x_i^T \boldsymbol x + b \rightarrow
   \sum_{i=1}^N \alpha_i  k(\boldsymbol x_i, \boldsymbol x) + b
$$

So we need to learn the $\alpha_i$ coefficients instead of the weights $\boldsymbol w$.

This can lead to very flexible models which have the main disadvantage not to scale well with the number of training points and as thus should be kept for small to medium size datasets.

Scikit-learn proposes at least two algorithm for regression based on kernel methods:

* Kernel Ridge Regression
* SVR (support vector regression)

##### Kernel Ridge Regression

Kernel Ridge regression is the application of the kernel trick to the Ridge regression algorithm.


Let's illustrate this with a toy function:

In [ ]:
x = np.linspace(-5, 5, 101)
y = np.sin(x) + 0.5 * x - 2
plt.plot(x, y)

The Scikit-learn implementation allows us to use several kernel functions. The most used (after the linear kernel) is the RBF kernel:

$$k_{rbf}(\boldsymbol x, \boldsymbol y) = e^{-\gamma \|\boldsymbol x - \boldsymbol y\|^2}$$

In [ ]:
from sklearn.kernel_ridge import KernelRidge

xtrain = x[5::10]
ftrain = y[5::10]
np.random.seed(4)
ytrain = ftrain + 0.5 * np.random.randn(len(ftrain))

xy_dict = dict(zip(xtrain, ytrain))

reg = KernelRidge(
    alpha=0.05,
    kernel="rbf",
    gamma=0.1,
)
reg.fit(xtrain[:, np.newaxis], ytrain)

plt.plot(x, y)
plt.plot(xtrain, ytrain, "o")
ypred = reg.predict(x[:, np.newaxis])
plt.plot(x, ypred)

print(r2_score(y, ypred))

##### Support Vector Regression

Support vector regression optimizes a $\epsilon$-tolerant loss function:

$$ \min_{\boldsymbol w, b}\ \mathcal R(\boldsymbol w) + C \sum_{i=1}^N l_\epsilon(\boldsymbol w^T \boldsymbol x_i + b - y_i) $$

where $\mathcal R(.)$ is a regularization term and $l_\epsilon(.)$ looks like:

In [ ]:
import numpy as np

In [ ]:
x = np.linspace(-5, 5, 101)
epsilon = 1
plt.plot(x, np.maximum(0, np.abs(x) - epsilon))

The flat section as for consequence that points which fall in this part do not contribute do the final solution. Only points on the edge of the tolerance tube and beyond do, hence the "support vector" name.

In [ ]:
from sklearn.svm import SVR

xy_dict = dict(zip(xtrain, ytrain))

epsilon = 0.5
reg = SVR(C=1e1, kernel="rbf", gamma=1e-1, epsilon=epsilon)
reg.fit(xtrain[:, np.newaxis], ytrain)

plt.plot(x, y)
plt.plot(xtrain, ytrain, "o")
ypred = reg.predict(x[:, np.newaxis])
plt.plot(x, ypred)
plt.fill_between(x, ypred + epsilon, ypred - epsilon, color="lightgray")

sv = reg.support_vectors_.flatten()
print(f"Number of support vectors: {len(sv)}")
plt.scatter(sv, [xy_dict[xi] for xi in sv], s=100, color="none", edgecolor="black")

print(r2_score(y, ypred))

**Exercise**: Play we the three main parameters and try to understand their effect.

* Higher `C` forces to pass through the points
* Lower values `gamma` leads to more slowly varying solutions
* `epsilon` directly sets the widths of the tolerance tube.

#### Multi-layer perceptron (MLP)

The perceptron model is just a linear transformation followed by an element-wise non-linear transformation:

$$ X_{out} = a(X_{in} W + b \boldsymbol 1) $$

Where $W$ is a $d_{in} \times d_{out}$ matrix.

The function $a(.)$ is typically the sigmoid function or the ReLU (rectified linear unit) function:

In [ ]:
x = np.linspace(-2, 2, 101)
plt.plot(x, np.tanh(x), label="sigmoid")
plt.plot(x, np.maximum(0, x), label="ReLU")
plt.legend()

The multi-layer perceptron model is merely a succession of several perceptron layers:

$$ X_{out} = (a(a(X_{in}W_1 + b_1)W_2 + b_2) \dots)W_L + b_L$$

The activation function of the last layer is often set as the identity function so that the output can scale to the output values.

So we can see a MLP as a non linear mapping $f_{mlp}(.)$ from a $d_{in}$ dimensional space to a $d_{out}$ dimensional space.

This gives a very flexible family of functions.

$$ \min_{W_1, \dots, W_L, b_1, \dots, b_L} \frac{1}{2N}\|f_{mlp}(X_{in}) - \boldsymbol y\|^2$$

One of the main disadvantage of the MLP model is that it can require a large number of training samples to be fit without over-fitting. The convergence is also very dependent on the initial values (which are usually set randomly).

For regular tabular data with heterogeneous inputs, tree based ensemble algorithms usually give better results.

### Regularization

We have presented regularization in the context of linear models. It should be noted that regularization, under different forms, is widely used in many different models. For instance:

* Some solvers used to train neural networks have a "decay" parameter (it may have other names) in which corresponds to $L_2$-regularization.
* XGBoost uses regularization terms on the function values which can be set through the `reg_alpha` ($L_1$) and `reg_lambda` ($L_2$) parameters. However, those parameters are usually left at their default values.
* Regularization is an important component of kernel models since the presence of the regularization term implies the applicability of the [**representer theorem**](https://en.wikipedia.org/wiki/Representer_theorem) and thus the usability of the [**kernel trick**](https://en.wikipedia.org/wiki/Kernel_method).

### Prediction intervals (Quantile regression)

The Scikit-learn implementation of Gradient Boosting allows us to perform quantile regression. This can be used to compute prediction intervals:

In [ ]:
exclude_columns = ["price", "tax_mpg_missing"]

y = df_cars.price.values
X = df_cars.drop(exclude_columns, axis=1)

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=1)

In [ ]:
from catboost import CatBoostRegressor

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.base import clone

cat_enc = TargetEncoder(cols=cat_cols)

reg = CatBoostRegressor(
    loss_function="MultiQuantile:alpha=0.05,0.5,0.95",
    n_estimators=400,
    depth=8,
    learning_rate=0.5,
    random_state=1,
)

model = Pipeline([("cat_enc", cat_enc), ("impute", IterativeImputer()), ("reg", reg)])

model.fit(Xdev, ydev)
ypred_low, ypred, ypred_up = model.predict(Xtest).clip(0, np.inf).T

car_pricing.plot_pred_errors(ytest, ypred)
_ = (
    pd.DataFrame({"importance": model["reg"].feature_importances_}, index=Xtest.columns)
    .sort_values("importance")
    .plot.barh()
)

In [ ]:
isort = np.argsort(ytest)

stride = 100

plt.figure(figsize=(12, 4))
plt.plot(ytest[isort][::stride], "--", zorder=10)
plt.plot(ypred[isort][::stride], lw=0.5)
_ = plt.fill_between(
    range(len(ytest[::stride])),
    ypred_up[isort][::stride],
    ypred_low[isort][::stride],
    color="lightgray",
)

### Bayesian optimization

Bayesian optimization techniques usually try to build an approximation of the function that links the hyper-parameters and the performance score. They are able to provide for a given point in the hyper-parameter space a predictive distribution (i.e. a distribution of the score). Based on this distribution they are able to compute the chance that a given hyper-parameter combination leads to an improvement of the score.

An improvement is usually possible in two configurations:

* the hyper-parameter combination lies in an area where the score is already good
* the hyper-parameter combination is far from already tested combinations and the score predictive distribution is wide.

This leads to an exploration of the hyper-parameter space which is expected to be more efficient than a  completely random exploration.

In [ ]:
from optuna.integration import OptunaSearchCV
from optuna.distributions import FloatDistribution, IntDistribution

from sklearn.utils import check_random_state

random_state = check_random_state(0)

model = CatBoostRegressor(cat_features=cat_cols, verbose=False, random_seed=1)

param_distr = {
    "n_estimators": IntDistribution(100, 499),  # uniformly picked between 100 and 499
    "depth": IntDistribution(3, 10),  # uniformly picked between 3 and 10
    "learning_rate": FloatDistribution(0, 1),  # Uniformly drawn in the [0 1] interval
}

cv = KFold(3, shuffle=True, random_state=random_state)
cv_model = OptunaSearchCV(
    model,
    max_iter=100,
    timeout=10 * 60.0,
    param_distributions=param_distr,
    random_state=random_state,
    scoring=r2_scorer,
    cv=cv,
    n_jobs=2,
)
_ = cv_model.fit(Xdev, ydev)

In [ ]:
cv_model.best_params_

In [ ]:
ypred = cv_model.predict(Xtest).clip(0, np.inf)
car_pricing.plot_pred_errors(ytest, ypred)
result = permutation_importance(cv_model, Xtest, ytest)
_ = (
    pd.DataFrame({"importance": result["importances_mean"]}, index=Xtest.columns)
    .sort_values("importance")
    .plot.barh()
)